In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
class Classifier_INCEPTION:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                                      min_lr=0.0001)

        file_path = self.output_directory + 'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model




In [ ]:
model = Classifier_INCEPTION("output", (137, 15), 5)

In [ ]:
f = np.load("mfcc.npz")
X, Y = f['X'], f['Y']
X_train =  X[100:1000]
Y_train =  Y[100:1000]            # fold 10


x_test, y_test = X[0:100], Y[0:100]
x_train, y_train = X_train[0:800], Y_train[0:800]
x_val, y_val= X_train[800:900], Y_train[800:900]

In [ ]:
mcp = keras.callbacks.ModelCheckpoint("/content/ckpt", "val_loss", save_best_only=True, save_weights_only=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                              min_lr=0.0001)
callbacks = [mcp, reduce_lr]
model.model.fit(x_train, y_train, batch_size=64, epochs=2000, validation_data=(x_val, y_val), callbacks=callbacks)

Epoch 1/2000
13/13 [==============================] - 26s 135ms/step - loss: 0.7815 - accuracy: 0.7613 - val_loss: 85.2701 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 2/2000
13/13 [==============================] - 0s 23ms/step - loss: 0.1698 - accuracy: 0.9488 - val_loss: 127.2391 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 3/2000
13/13 [==============================] - 0s 23ms/step - loss: 0.1094 - accuracy: 0.9650 - val_loss: 121.2421 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 4/2000
13/13 [==============================] - 0s 39ms/step - loss: 0.0710 - accuracy: 0.9812 - val_loss: 26.8272 - val_accuracy: 0.2900 - lr: 0.0010
Epoch 5/2000
13/13 [==============================] - 0s 37ms/step - loss: 0.0621 - accuracy: 0.9762 - val_loss: 25.2936 - val_accuracy: 0.2100 - lr: 0.0010
Epoch 6/2000
13/13 [==============================] - 0s 37ms/step - loss: 0.0631 - accuracy: 0.9812 - val_loss: 16.7109 - val_accuracy: 0.2000 - lr: 0.0010
Epoch 7/2000
13/13 [==============================] - 

In [ ]:
model.model.load_weights("/content/ckpt")

In [ ]:
y_pred = model.model(x_test)

In [ ]:
acc_metric = keras.metrics.SparseCategoricalAccuracy()
acc_metric.reset_states()
acc_metric.update_state(y_test, y_pred)
test_acc = acc_metric.result().numpy()

In [ ]:
test_acc

1.0

In [ ]:
np.save("fold.npy", y_pred.numpy())